# Project Goal: 
Extract YELP reviews for top 100 businesses in San Francisco and use bag-of-words to find the top words used in total reviews for each business.

In [1]:
import numpy as np
import pandas as pd
import requests
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

In [2]:
HTML_list = []
for i in np.arange(0,110,10):
    url = 'https://www.yelp.com/search?find_loc=San+Francisco,+CA,+USA&start=%s&cflt=restaurants' %i
    #print url
    response = requests.get(url)
    HTML_list.append(response.text)

results_list = []
for page in HTML_list:
    results = Selector(text=page).xpath("//span[@class='indexed-biz-name']/a[@class='biz-name js-analytics-click']/span")
    results_list.append(results)

In [103]:
name_list = []
for result in results_list:
        for name in result:
            name_list.append(name.xpath("text()").extract())

In [104]:
name_list[10]

[u'Brenda\u2019s French Soul Food']

In [105]:
name_list = pd.Series(name_list)
name_list = name_list.apply(lambda x: x[0].encode('ascii', 'ignore').rstrip().lstrip().replace(' ','-').replace('.','').replace("\u2019",''))

In [106]:
name_list

0                              Ichido
1                             Dakshin
2                        b-Patisserie
3                    Cuisine-of-Nepal
4                              Bodega
5                         Casa-Flores
6                             Tacorea
7                             Frances
8                           Sugarfoot
9               State-Bird-Provisions
10           Brendas-French-Soul-Food
11                               Nopa
12                          Buzzworks
13                         Kebab-King
14                            Paprika
15        Chicks-and-Love-Pizza-Patio
16           Italian-Homemade-Company
17                             Bodega
18                        El-Farolito
19                       Rove-Kitchen
20                       No-No-Burger
21                                Lol
22                            Ro-Cafe
23                            Pokihub
24                         Kebab-King
25                            Hogwash
26          

In [144]:
# Please only run this frame once to avoid hitting the site too hard all at once
def yelp_func(search_key):
    import requests

    # Search parameters -- for convenience 

    # Search and location get replaced in the next few lines

    HTML_list = []

    for i in ["?start=%d" %d for d in range(0,140,20)]:
        url = search_key+"%s" %i
        #print url
        response = requests.get(url)
        HTML_list.append(response.text)

    reviews_list = []
    from scrapy.selector import Selector
    from scrapy.http import HtmlResponse

    results_list = []
    for page in HTML_list:
    # This xpath query will find the number of total results
        results = Selector(text=page).xpath("//div[@class='review-wrapper']/div[@class='review-content']/p")
        results_list.append(results)

    for result in results_list:
        for review in result:
            reviews_list.append(review.xpath("text()").extract())
    #total_results = int(results[0].extract())
    #total_results


    southern_pacific = []
    for i in reviews_list:
        southern_pacific.append(i)

    import re
    from nltk.corpus import stopwords # Import the stop word list

    def review_to_words( raw_review ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)

        letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 
        #
        # 3. Convert to lower case, split into individual words
        words = letters_only.lower().split()                             
        #
        # 4. In Python, searching a set is much faster than searching
        #   a list, so convert the stop words to a set

        stops = set(stopwords.words("english"))                  
        # 
        # 5. Remove stop words
        meaningful_words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( meaningful_words ))  

    clean_train_reviews = []
    for i in southern_pacific:
        clean_train_reviews.append(review_to_words(str(i)))

    print "Creating the bag of words...\n"
    from sklearn.feature_extraction.text import CountVectorizer

    # Initialize the "CountVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 5000) 

    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of 
    # strings.
    train_data_features = vectorizer.fit_transform(clean_train_reviews)

    # Numpy arrays are easy to work with, so convert the result to an 
    # array
    train_data_features = train_data_features.toarray()

    # Take a look at the words in the vocabulary
    vocab = vectorizer.get_feature_names()


    import numpy as np

    # Sum up the counts of each vocabulary word
    dist = np.sum(train_data_features, axis=0)

    # For each, print the vocabulary word and the number of times it 
    # appears in the training set

    count_list = []
    word_list = []
    for tag, count in zip(vocab, dist):
        #print count, tag
        count_list.append(count)
        word_list.append(tag)

    import pandas as pd
    word_df = pd.DataFrame({'word': word_list,'count': count_list})
    
    return word_df


In [145]:
cloud_list_dict = {}
for i in name_list:
    try:
        cloud_list_dict[i] = yelp_func('https://www.yelp.com/biz/%s-san-francisco' %i)
    except: 
        continue
      

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating the bag of words...

Creating t

In [146]:
cloud_list_dict['Ichido']

,count,word
0,1,ability
1,1,abroad
2,1,absolute
3,4,absolutely
4,1,accommodating
5,1,accompany
6,1,according
7,1,accordingly
8,1,acidity
9,1,actual


In [147]:
for i in name_list:
    print 'https://www.yelp.com/biz/%s-san-francisco' %i

https://www.yelp.com/biz/Ichido-san-francisco
https://www.yelp.com/biz/Dakshin-san-francisco
https://www.yelp.com/biz/b-Patisserie-san-francisco
https://www.yelp.com/biz/Cuisine-of-Nepal-san-francisco
https://www.yelp.com/biz/Bodega-san-francisco
https://www.yelp.com/biz/Casa-Flores-san-francisco
https://www.yelp.com/biz/Tacorea-san-francisco
https://www.yelp.com/biz/Frances-san-francisco
https://www.yelp.com/biz/Sugarfoot-san-francisco
https://www.yelp.com/biz/State-Bird-Provisions-san-francisco
https://www.yelp.com/biz/Brendas-French-Soul-Food-san-francisco
https://www.yelp.com/biz/Nopa-san-francisco
https://www.yelp.com/biz/Buzzworks-san-francisco
https://www.yelp.com/biz/Kebab-King-san-francisco
https://www.yelp.com/biz/Paprika-san-francisco
https://www.yelp.com/biz/Chicks-and-Love-Pizza-Patio-san-francisco
https://www.yelp.com/biz/Italian-Homemade-Company-san-francisco
https://www.yelp.com/biz/Bodega-san-francisco
https://www.yelp.com/biz/El-Farolito-san-francisco
https://www.yelp

In [148]:
new_cloud = cloud_list_dict

In [149]:
new_cloud['Ichido']

,count,word
0,1,ability
1,1,abroad
2,1,absolute
3,4,absolutely
4,1,accommodating
5,1,accompany
6,1,according
7,1,accordingly
8,1,acidity
9,1,actual


In [181]:
# extract 20 top words
short_cloud = {}
for i in new_cloud:
    try:
        new_cloud[i] = new_cloud[i].set_index('word').drop('xa').reset_index().sort_values('count', ascending = False)   
        short_cloud = new_cloud[i][0:20]
    except:
        new_cloud[i] = new_cloud[i].sort_values('count', ascending = False)
        short_cloud[i] = new_cloud[i][0:20]


In [183]:
short_cloud.keys()

['20-Spot',
 'Parada-22',
 'Stuffed',
 'Nopa',
 'Two-Sons-Sandwiches',
 'Reems',
 'An-Japanese-Restaurant',
 'State-Bird-Provisions',
 'Buzzworks',
 'Bodega',
 'Garaje',
 'Sugarfoot',
 'Souvla',
 'HRD',
 'Prubechu',
 'Kitchen-Istanbul',
 'Birba',
 'Cafe-de-Casa',
 'Garajito',
 'Bite',
 'Brendas-French-Soul-Food',
 'Azucar-Lounge',
 'Wooly-Pig-Cafe',
 'Kokkari-Estiatorio',
 'Universal-Cafe',
 'Hopwater-Distribution',
 'Bouche',
 'Hogwash',
 'Italian-Homemade-Company',
 'The-Codmother-Fish-and-Chips',
 'Cuisine-of-Nepal',
 'The-House',
 'DragonEats',
 'El-Farolito',
 'Zazie',
 'Farmhouse-Kitchen-Thai-Cuisine',
 'Tacorea',
 'Plow',
 'Lol',
 'Elephant-Sushi',
 'Bayshore-Taqueria',
 'Rove-Kitchen',
 'Dotties-True-Blue-Cafe',
 'Sotto-Mare',
 'Dancing-Bull',
 'Stones-Throw',
 'Junior-Barbecue-Burger',
 'The-Spice-Jar',
 'The-Chairman',
 'Chicks-and-Love-Pizza-Patio',
 'Nightbird',
 'Tasty-Pot',
 'Paprika',
 'No-No-Burger',
 'Fat-Angel',
 'Kebab-King',
 'Lavash',
 'Old-Jerusalem-Restaurant',
 

In [211]:
# create initial df
final_cloud_df = pd.DataFrame({'Ichido' : short_cloud['Ichido']['word'],'Ichido_count' : short_cloud['Ichido']['count']})

In [212]:
final_cloud_df.shape

(20, 2)

In [213]:
# append to df
for i in short_cloud:
    if i != 'Ichido':
        #short_cloud[i].sort_values('count', ascending = False, inplace = True)
        print i, np.array(short_cloud[i]['word'])[0:5], pd.Series(short_cloud[i]['count'])[0:5]
        final_cloud_df[i] = short_cloud[i]['word'].tolist()
        final_cloud_df[i+'_count'] = short_cloud[i]['count'].tolist()
    else:
        continue

20-Spot [u'wine' u'place' u'spot' u'great' u'food'] 2124    186
1376    103
1806     99
816      94
732      80
Name: count, dtype: int64
Parada-22 [u'food' u'puerto' u'good' u'place' u'rican'] 740     326
1508    186
825     176
1424    165
1613    149
Name: count, dtype: int64
Stuffed [u'pierogi' u'food' u'place' u'pierogis' u'good'] 1279    121
658     109
1290    107
1282     75
734      75
Name: count, dtype: int64
Nopa [u'pork' u'good' u'food' u'nopa' u'chop'] 2054    137
1193    135
1091    129
1793    124
502      93
Name: count, dtype: int64
Two-Sons-Sandwiches [u'sandwich' u'sandwiches' u'great' u'place' u'good'] 1077    128
1078    113
535      77
933      67
527      47
Name: count, dtype: int64
Reems [u'reem' u'food' u'man' u'bread' u'market'] 545    32
255    28
391    19
81     19
401    18
Name: count, dtype: int64
An-Japanese-Restaurant [u'sushi' u'omakase' u'nigiri' u'fish' u'chef'] 1443    109
981      86
950      62
515      44
219      44
Name: count, dtype: int64


In [214]:
final_cloud_df

,Ichido,Ichido_count,20-Spot,20-Spot_count,Parada-22,Parada-22_count,Stuffed,Stuffed_count,Nopa,Nopa_count,...,Street-Taco,Street-Taco_count,Pokihub,Pokihub_count,FOB-Kitchen,FOB-Kitchen_count,Lous-Cafe,Lous-Cafe_count,La-Taqueria,La-Taqueria_count
482,geoff,56,wine,186,food,326,pierogi,121,pork,137,...,tacos,117,poke,63,food,87,sandwich,191,burrito,258
379,experience,46,place,103,puerto,186,food,109,good,135,...,place,89,place,58,filipino,69,sandwiches,136,tacos,140
682,meal,37,spot,99,good,176,place,107,food,129,...,good,85,bowl,55,fob,48,lou,103,food,139
446,food,37,great,94,place,165,pierogis,75,nopa,124,...,food,83,fresh,46,rice,47,good,76,place,124
297,dinner,35,food,80,rican,149,good,75,chop,93,...,burrito,75,fish,38,kitchen,43,place,72,taco,122
277,delicious,34,good,76,rice,132,like,73,table,93,...,taco,71,spicy,35,lumpia,43,sauce,71,taqueria,113
763,one,33,bar,71,plantains,130,one,66,like,91,...,meat,50,good,34,good,38,time,65,super,110
176,chef,32,like,57,beans,123,great,54,one,89,...,street,50,salmon,34,place,31,get,62,one,107
225,course,28,menu,47,great,120,back,52,time,87,...,salsa,45,rice,34,like,30,bread,58,good,105
29,amazing,26,little,45,cha,110,stuffed,52,great,79,...,one,45,really,29,great,28,line,53,la,103


In [216]:
final_cloud_df.to_csv('./cloud_yelp_csv.csv')

In [218]:
final_cloud_df.shape

(20, 176)